In [1]:
#Imports
import pandas as pd
import numpy as np
from numpy import nan
import json
from textblob import TextBlob
import pandas.io.json as pd_json
import re

# Proceso ETL y aplicacion de analisis de sentimiento 🤺


In [2]:
#Se leen los datos del Json australian_user_reviews.json y se importan a un dataframe

rows = []

with open('australian_user_reviews.json', encoding='utf-8') as f:
    for line in f:
        rows.append(eval(line))

df_user = pd.DataFrame(rows)


In [3]:
#Se borran los NaN y los duplicados
df_user = df_user.dropna()

In [4]:
#Lectura del dataframe

df_user

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [5]:
#Se leen los datos del Json australian_user_items.json y se importan a un dataframe

rows = []

with open('australian_users_items.json', encoding='utf-8') as f:
    for line in f:
        rows.append(eval(line))

df_items = pd.DataFrame(rows)

In [6]:
#Lectura del dataframe

df_items

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


In [7]:
#Se leen los datos del Json output_steam_games.json con ast y se importan a un dataframe

rows = []

with open('output_steam_games.json', encoding='utf-8') as f:
    for line in f:
        rows.append(json.loads(line))

df_games = pd.DataFrame(rows)

In [8]:
#Eliminar los NaNs del dataframe

df_games = df_games.dropna()


In [9]:
#Lectura del dataframe

df_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域
88315,Trickjump Games Ltd,"[Action, Adventure, Simulation]",Battle Royale Trainer,Battle Royale Trainer,http://store.steampowered.com/app/772540/Battl...,2018-01-04,"[Action, Adventure, Simulation, FPS, Shooter, ...",http://steamcommunity.com/app/772540/reviews/?...,"[Single-player, Steam Achievements]",3.99,False,772540,Trickjump Games Ltd
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120439,Bidoniera Games,"[Action, Adventure, Casual, Indie]",Kebab it Up!,Kebab it Up!,http://store.steampowered.com/app/745400/Kebab...,2018-01-04,"[Action, Indie, Casual, Violent, Adventure]",http://steamcommunity.com/app/745400/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",1.99,False,745400,Bidoniera Games
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich


## ETL USER_REVIEWS 🤺


In [10]:
#Desanidamos la columna reviews del dataframe df_user usando explode

df_user = df_user.explode('reviews')


In [11]:
#Lectura del dataframe

df_user

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20..."
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011...."
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011..."
1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014...."
1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2..."
...,...,...,...
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 10.', 'la..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 8.', 'las..."
25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '1 person found this review funny', ..."
25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '', 'posted': 'Posted July 20.', 'la..."


In [12]:
#Se normaliza la columna reviews para trabajar con ella

df_user2 = pd.json_normalize(df_user['reviews'])

In [13]:
#Lectura del dataframe
df_user2

,funny,posted,last_edited,item_id,helpful,recommend,review
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...
59328,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...
59329,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...
59330,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
59331,,Posted July 20.,,730,No ratings yet,True,:D


In [14]:
#Creacion de la columna sentiment_analysis

df_user2['sentiment_analysis'] = ""



In [15]:
df_user2.dtypes

funny                 object
posted                object
last_edited           object
item_id               object
helpful               object
recommend             object
review                object
sentiment_analysis    object
dtype: object

In [16]:
#Se aplica la funcion sentiment_analysis a la columna review y se hace el display del mismo en sentiment_analysis

df_user2['sentiment_analysis'] = df_user2['review'].astype(str).apply(lambda x: TextBlob(x).sentiment.polarity)


In [17]:
#Lectura del dataframe

df_user2

,funny,posted,last_edited,item_id,helpful,recommend,review,sentiment_analysis
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,0.174444
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,0.337500
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,0.050000
3,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,0.114583
4,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,-0.036111
...,...,...,...,...,...,...,...,...
59328,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...,0.233333
59329,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...,0.142500
59330,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...,0.087500
59331,,Posted July 20.,,730,No ratings yet,True,:D,1.000000


In [18]:
#Se modifican para que los valores sean "0" si es malo y "1" si es neutral y "2" si es bueno

df_user2['sentiment_analysis'] = df_user2['sentiment_analysis'].apply(lambda x: 0 if x < 0 else (1 if x == 0 else 2))

In [19]:
#Lectura del dataframe

df_user2

,funny,posted,last_edited,item_id,helpful,recommend,review,sentiment_analysis
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,2
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,2
3,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,2
4,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,0
...,...,...,...,...,...,...,...,...
59328,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...,2
59329,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...,2
59330,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...,2
59331,,Posted July 20.,,730,No ratings yet,True,:D,2


In [20]:
#Unimos los dos dataframes df_user y df_user2 usando merge en un nuevo dataframe df_userfinal

df_userfinal = pd.merge(df_user, df_user2, left_index=True, right_index=True)

In [21]:
#Lectura del dataframe

df_userfinal

,user_id,user_url,reviews,funny,posted,last_edited,item_id,helpful,recommend,review,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20...",,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011....",,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011...",,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2
1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014....",,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,2
1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2...",,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,2
...,...,...,...,...,...,...,...,...,...,...,...
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 10.', 'la...",,Posted June 2.,,209870,0 of 3 people (0%) found this review helpful,True,blackman retribution,1
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 8.', 'las...",,Posted June 2.,,209870,0 of 3 people (0%) found this review helpful,True,blackman retribution,1
25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '1 person found this review funny', ...",,Posted April 19.,Last edited May 20.,437220,0 of 1 people (0%) found this review helpful,True,เรื่องของภาพ 10/10 อาวุธ 8/10 ความlag 1/10 ...,1
25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '', 'posted': 'Posted July 20.', 'la...",,Posted April 19.,Last edited May 20.,437220,0 of 1 people (0%) found this review helpful,True,เรื่องของภาพ 10/10 อาวุธ 8/10 ความlag 1/10 ...,1


In [22]:
#Se borra la columna review y reviews del dataframe df_userfinal

df_userfinal = df_userfinal.drop(['review'], axis=1)
df_userfinal = df_userfinal.drop(['reviews'], axis=1)

In [23]:
#Lectura del dataframe

df_userfinal

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,2
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,2
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,2
1,js41637,http://steamcommunity.com/id/js41637,,"Posted July 15, 2011.",,22200,No ratings yet,True,2
1,js41637,http://steamcommunity.com/id/js41637,,"Posted July 15, 2011.",,22200,No ratings yet,True,2
...,...,...,...,...,...,...,...,...,...
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,,Posted June 2.,,209870,0 of 3 people (0%) found this review helpful,True,1
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,,Posted June 2.,,209870,0 of 3 people (0%) found this review helpful,True,1
25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,,Posted April 19.,Last edited May 20.,437220,0 of 1 people (0%) found this review helpful,True,1
25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,,Posted April 19.,Last edited May 20.,437220,0 of 1 people (0%) found this review helpful,True,1


In [24]:
#Se borran las filas que tengan NaNs

df_userfinal = df_userfinal.dropna(how="all")

In [25]:
#Lectura del dataframe

df_userfinal

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,2
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,2
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,2
1,js41637,http://steamcommunity.com/id/js41637,,"Posted July 15, 2011.",,22200,No ratings yet,True,2
1,js41637,http://steamcommunity.com/id/js41637,,"Posted July 15, 2011.",,22200,No ratings yet,True,2
...,...,...,...,...,...,...,...,...,...
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,,Posted June 2.,,209870,0 of 3 people (0%) found this review helpful,True,1
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,,Posted June 2.,,209870,0 of 3 people (0%) found this review helpful,True,1
25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,,Posted April 19.,Last edited May 20.,437220,0 of 1 people (0%) found this review helpful,True,1
25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,,Posted April 19.,Last edited May 20.,437220,0 of 1 people (0%) found this review helpful,True,1


## ETL USER_ITEMS 🤺


In [26]:
#Desanidamos la columna items del dataframe df_items usando explode
df_items = df_items.explode('items')


In [27]:
#Lectura del dataframe
df_items

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '10', 'item_name': 'Counter-Strike..."
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '20', 'item_name': 'Team Fortress ..."
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '30', 'item_name': 'Day of Defeat'..."
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '40', 'item_name': 'Deathmatch Cla..."
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '50', 'item_name': 'Half-Life: Opp..."
...,...,...,...,...,...
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '373330', 'item_name': 'All Is Dus..."
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '388490', 'item_name': 'One Way To..."
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '521570', 'item_name': 'You Have 1..."
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '519140', 'item_name': 'Minds Eyes..."


In [28]:
df_items.columns

Index(['user_id', 'items_count', 'steam_id', 'user_url', 'items'], dtype='object')

In [29]:
#Se normaliza la columna items para trabajar con ella

df_items2 = pd.json_normalize(df_items['items'])


In [30]:
#Lectura del dataframe
df_items2

,item_id,item_name,playtime_forever,playtime_2weeks
0,10,Counter-Strike,6.0,0.0
1,20,Team Fortress Classic,0.0,0.0
2,30,Day of Defeat,7.0,0.0
3,40,Deathmatch Classic,0.0,0.0
4,50,Half-Life: Opposing Force,0.0,0.0
...,...,...,...,...
5170010,373330,All Is Dust,0.0,0.0
5170011,388490,One Way To Die: Steam Edition,3.0,3.0
5170012,521570,You Have 10 Seconds 2,4.0,4.0
5170013,519140,Minds Eyes,3.0,3.0


In [31]:
#Unimos los dos dataframes df_items y df_items2 en uno solo llamado df_itemsfinal con merge

df_itemsfinal = pd.merge(df_items, df_items2, left_index=True, right_index=True)

In [32]:
#Lectura del dataframe

df_itemsfinal

,user_id,items_count,steam_id,user_url,items,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '10', 'item_name': 'Counter-Strike...",10,Counter-Strike,6.0,0.0
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '20', 'item_name': 'Team Fortress ...",10,Counter-Strike,6.0,0.0
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '30', 'item_name': 'Day of Defeat'...",10,Counter-Strike,6.0,0.0
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '40', 'item_name': 'Deathmatch Cla...",10,Counter-Strike,6.0,0.0
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '50', 'item_name': 'Half-Life: Opp...",10,Counter-Strike,6.0,0.0
...,...,...,...,...,...,...,...,...,...
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '373330', 'item_name': 'All Is Dus...",227220,Sang-Froid - Tales of Werewolves,2973.0,0.0
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '388490', 'item_name': 'One Way To...",227220,Sang-Froid - Tales of Werewolves,2973.0,0.0
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '521570', 'item_name': 'You Have 1...",227220,Sang-Froid - Tales of Werewolves,2973.0,0.0
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '519140', 'item_name': 'Minds Eyes...",227220,Sang-Froid - Tales of Werewolves,2973.0,0.0


In [33]:
#Se borra la columna items del dataframe df_itemsfinal

df_itemsfinal = df_itemsfinal.drop(['items'], axis=1)


In [34]:
#Lectura del dataframe

df_itemsfinal

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6.0,0.0
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6.0,0.0
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6.0,0.0
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6.0,0.0
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6.0,0.0
...,...,...,...,...,...,...,...,...
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,227220,Sang-Froid - Tales of Werewolves,2973.0,0.0
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,227220,Sang-Froid - Tales of Werewolves,2973.0,0.0
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,227220,Sang-Froid - Tales of Werewolves,2973.0,0.0
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,227220,Sang-Froid - Tales of Werewolves,2973.0,0.0


In [35]:
#Se borran las filas que tengan NaNs

df_itemsfinal = df_itemsfinal.dropna(how="all")

In [36]:
#Lectura del dataframe

df_itemsfinal

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6.0,0.0
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6.0,0.0
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6.0,0.0
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6.0,0.0
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6.0,0.0
...,...,...,...,...,...,...,...,...
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,227220,Sang-Froid - Tales of Werewolves,2973.0,0.0
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,227220,Sang-Froid - Tales of Werewolves,2973.0,0.0
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,227220,Sang-Froid - Tales of Werewolves,2973.0,0.0
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,227220,Sang-Froid - Tales of Werewolves,2973.0,0.0


## ETL GAMES 🤺

In [37]:
#Lectura del dataframe

df_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域
88315,Trickjump Games Ltd,"[Action, Adventure, Simulation]",Battle Royale Trainer,Battle Royale Trainer,http://store.steampowered.com/app/772540/Battl...,2018-01-04,"[Action, Adventure, Simulation, FPS, Shooter, ...",http://steamcommunity.com/app/772540/reviews/?...,"[Single-player, Steam Achievements]",3.99,False,772540,Trickjump Games Ltd
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120439,Bidoniera Games,"[Action, Adventure, Casual, Indie]",Kebab it Up!,Kebab it Up!,http://store.steampowered.com/app/745400/Kebab...,2018-01-04,"[Action, Indie, Casual, Violent, Adventure]",http://steamcommunity.com/app/745400/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",1.99,False,745400,Bidoniera Games
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich


In [38]:
#Columnas del dataframe

df_games.columns

Index(['publisher', 'genres', 'app_name', 'title', 'url', 'release_date',
       'tags', 'reviews_url', 'specs', 'price', 'early_access', 'id',
       'developer'],
      dtype='object')

In [39]:
#Revisamos los datos unicos de cada columna

print(df_games['price'].unique())

[4.99 'Free To Play' 'Free to Play' 0.99 3.99 18.99 29.99 10.99 2.99 1.59
 14.99 1.99 59.99 9.99 6.99 7.99 39.99 'Free' 19.99 7.49 8.99 12.99 5.99
 2.49 15.99 1.25 24.99 17.99 61.99 3.49 11.99 13.99 'Free Demo'
 'Play for Free!' 34.99 1.49 32.99 99.99 14.95 69.99 16.99 79.99 49.99
 13.98 149.99 771.71 'Install Now' 21.99 89.99
 'Play WARMACHINE: Tactics Demo' 0.98 139.92 4.29 'Free Mod' 54.99 64.99
 74.99 0.89 0.5 'Play Now' 299.99 1.29 119.99 44.99 3.0 15.0 1.39
 'Free HITMAN™ Holiday Pack' 2.0 4.0 1.95 1.5 6.66 26.99 399.99 31.99 20.0
 40.0 5.0 3.33 38.85 71.7 995.0 5.49 27.49 3.39 6.0 19.95 20.99 499.99
 27.99 199.99 4.68 131.4 44.98 202.76 2.3 0.95 36.99 172.24 249.99 2.97
 10.96 2.66 6.48 10.0 1.0 11.15 'Play the Demo' 49.0 199.0 99.0 87.94 0.49
 9.98 9.95 12.89 6.49 1.87 41.99 4.49 289.99 23.96 5.65 12.0 129.99 23.99
 13.37 189.96 59.95 160.91 'Third-party']


In [40]:
#Suplantamos aquellas que digan free u otras por 0.00

df_games['price'] = df_games['price'].replace(['Free','','Play Now','Third-party','Play the Demo','Play WARMACHINE: Tactics Demo', 'Free To Play','Free HITMAN™ Holiday Pack', 'Free to Play', 'Play for Free!', 'Install Now', 'Free Demo', 'Free Mod', 'Free Movie', 'Free to Use'], '0.00')

In [41]:
#Revisamos los datos unicos de cada columna

print(df_games['price'].unique())

[4.99 '0.00' 0.99 3.99 18.99 29.99 10.99 2.99 1.59 14.99 1.99 59.99 9.99
 6.99 7.99 39.99 19.99 7.49 8.99 12.99 5.99 2.49 15.99 1.25 24.99 17.99
 61.99 3.49 11.99 13.99 34.99 1.49 32.99 99.99 14.95 69.99 16.99 79.99
 49.99 13.98 149.99 771.71 21.99 89.99 0.98 139.92 4.29 54.99 64.99 74.99
 0.89 0.5 299.99 1.29 119.99 44.99 3.0 15.0 1.39 2.0 4.0 1.95 1.5 6.66
 26.99 399.99 31.99 20.0 40.0 5.0 3.33 38.85 71.7 995.0 5.49 27.49 3.39
 6.0 19.95 20.99 499.99 27.99 199.99 4.68 131.4 44.98 202.76 2.3 0.95
 36.99 172.24 249.99 2.97 10.96 2.66 6.48 10.0 1.0 11.15 49.0 199.0 99.0
 87.94 0.49 9.98 9.95 12.89 6.49 1.87 41.99 4.49 289.99 23.96 5.65 12.0
 129.99 23.99 13.37 189.96 59.95 160.91]


In [42]:
#Revisamos los datos unicos de cada columna

print(df_games['release_date'].unique())
print("######################")
print(df_games['publisher'].unique())
print("######################")
print(df_games['app_name'].unique())
print("######################")
print(df_games['early_access'].unique())
print("######################")
print(df_games['developer'].unique())
print("######################")
print(df_games['id'].unique())

#Columnas como genres, tags etc no se pueden por el tipo de dato.

['2018-01-04' '2017-07-24' '2017-12-07' ... '2003-11-01' '2004-03-16'
 '2004-03-01']
######################
['Kotoshiro' 'Making Fun, Inc.' 'Poolians.com' ... 'OrtiGames/OrtiSoft'
 'INGAME' 'Bidoniera Games']
######################
['Lost Summoner Kitty' 'Ironbound' 'Real Pool 3D - Poolians' ...
 'LOGistICAL: South Africa' 'Russian Roads' 'EXIT 2 - Directions']
######################
[False True]
######################
['Kotoshiro' 'Secret Level SRL' 'Poolians.com' ...
 'Oscar Ortigueira López,OrtiGames/OrtiSoft' 'INGAME' 'Bidoniera Games']
######################
['761140' '643980' '670290' ... '733530' '610660' '658870']


In [43]:
#Se borran las filas que tengan NaNs

df_games = df_games.dropna(how="all")

In [44]:
#Lectura del dataframe

df_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,False,643980,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,False,670290,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域
88315,Trickjump Games Ltd,"[Action, Adventure, Simulation]",Battle Royale Trainer,Battle Royale Trainer,http://store.steampowered.com/app/772540/Battl...,2018-01-04,"[Action, Adventure, Simulation, FPS, Shooter, ...",http://steamcommunity.com/app/772540/reviews/?...,"[Single-player, Steam Achievements]",3.99,False,772540,Trickjump Games Ltd
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120439,Bidoniera Games,"[Action, Adventure, Casual, Indie]",Kebab it Up!,Kebab it Up!,http://store.steampowered.com/app/745400/Kebab...,2018-01-04,"[Action, Indie, Casual, Violent, Adventure]",http://steamcommunity.com/app/745400/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",1.99,False,745400,Bidoniera Games
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich


In [45]:
#Se crea una columna llamada year y solo se le asigna el año de la columna release_date

df_games['year'] = df_games['release_date'].astype(str).str[:4]

In [46]:
df_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,year
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,2018
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,False,643980,Secret Level SRL,2018
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,False,670290,Poolians.com,2017
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域,2017
88315,Trickjump Games Ltd,"[Action, Adventure, Simulation]",Battle Royale Trainer,Battle Royale Trainer,http://store.steampowered.com/app/772540/Battl...,2018-01-04,"[Action, Adventure, Simulation, FPS, Shooter, ...",http://steamcommunity.com/app/772540/reviews/?...,"[Single-player, Steam Achievements]",3.99,False,772540,Trickjump Games Ltd,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120439,Bidoniera Games,"[Action, Adventure, Casual, Indie]",Kebab it Up!,Kebab it Up!,http://store.steampowered.com/app/745400/Kebab...,2018-01-04,"[Action, Indie, Casual, Violent, Adventure]",http://steamcommunity.com/app/745400/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",1.99,False,745400,Bidoniera Games,2018
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS""",2018
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada,2018
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich,2018


In [47]:
#rellenamos los title que esten vacios con el nombre del app_name y viceversa

df_games['title'] = df_games['title'].fillna(df_games['app_name'])
df_games['app_name'] = df_games['app_name'].fillna(df_games['title'])

In [48]:
#Lectura del dataframe

df_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,year
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,2018
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,False,643980,Secret Level SRL,2018
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,False,670290,Poolians.com,2017
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域,2017
88315,Trickjump Games Ltd,"[Action, Adventure, Simulation]",Battle Royale Trainer,Battle Royale Trainer,http://store.steampowered.com/app/772540/Battl...,2018-01-04,"[Action, Adventure, Simulation, FPS, Shooter, ...",http://steamcommunity.com/app/772540/reviews/?...,"[Single-player, Steam Achievements]",3.99,False,772540,Trickjump Games Ltd,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120439,Bidoniera Games,"[Action, Adventure, Casual, Indie]",Kebab it Up!,Kebab it Up!,http://store.steampowered.com/app/745400/Kebab...,2018-01-04,"[Action, Indie, Casual, Violent, Adventure]",http://steamcommunity.com/app/745400/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",1.99,False,745400,Bidoniera Games,2018
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS""",2018
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada,2018
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich,2018


In [49]:
#Borramos la columna app_name

df_games = df_games.drop(['app_name'], axis=1)

In [50]:
#Lectura del dataframe

df_games

,publisher,genres,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,year
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,2018
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,False,643980,Secret Level SRL,2018
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,False,670290,Poolians.com,2017
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域,2017
88315,Trickjump Games Ltd,"[Action, Adventure, Simulation]",Battle Royale Trainer,http://store.steampowered.com/app/772540/Battl...,2018-01-04,"[Action, Adventure, Simulation, FPS, Shooter, ...",http://steamcommunity.com/app/772540/reviews/?...,"[Single-player, Steam Achievements]",3.99,False,772540,Trickjump Games Ltd,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120439,Bidoniera Games,"[Action, Adventure, Casual, Indie]",Kebab it Up!,http://store.steampowered.com/app/745400/Kebab...,2018-01-04,"[Action, Indie, Casual, Violent, Adventure]",http://steamcommunity.com/app/745400/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",1.99,False,745400,Bidoniera Games,2018
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS""",2018
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada,2018
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich,2018


In [51]:
#Miramos las columnas de todos los dataframes para examinar como unificar todos los dataframes

print(df_userfinal.columns)
print("######################")
print(df_itemsfinal.columns)
print("######################")
print(df_games.columns)


Index(['user_id', 'user_url', 'funny', 'posted', 'last_edited', 'item_id',
       'helpful', 'recommend', 'sentiment_analysis'],
      dtype='object')
######################
Index(['user_id', 'items_count', 'steam_id', 'user_url', 'item_id',
       'item_name', 'playtime_forever', 'playtime_2weeks'],
      dtype='object')
######################
Index(['publisher', 'genres', 'title', 'url', 'release_date', 'tags',
       'reviews_url', 'specs', 'price', 'early_access', 'id', 'developer',
       'year'],
      dtype='object')


In [52]:
df_itemsfinal.head(5)

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6.0,0.0
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6.0,0.0
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6.0,0.0
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6.0,0.0
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6.0,0.0


In [53]:
df_userfinal.head(5)

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,2
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,2
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,2
1,js41637,http://steamcommunity.com/id/js41637,,"Posted July 15, 2011.",,22200,No ratings yet,True,2
1,js41637,http://steamcommunity.com/id/js41637,,"Posted July 15, 2011.",,22200,No ratings yet,True,2


In [54]:
#Buscamos dentro de df_games en la columna "id" el "1250" o "10"

df_games[df_games['id'].isin(['1250','10'])]



,publisher,genres,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,year
119997,Tripwire Interactive,[Action],Killing Floor,http://store.steampowered.com/app/1250/Killing...,2009-05-14,"[FPS, Zombies, Co-op, Survival, Action, Multip...",http://steamcommunity.com/app/1250/reviews/?br...,"[Single-player, Multi-player, Co-op, Cross-Pla...",19.99,False,1250,Tripwire Interactive,2009
120416,Valve,[Action],Counter-Strike,http://store.steampowered.com/app/10/CounterSt...,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...",http://steamcommunity.com/app/10/reviews/?brow...,"[Multi-player, Valve Anti-Cheat enabled]",9.99,False,10,Valve,2000


In [55]:
#Nos damos cuenta de que user_id y user_id son los mismos en los dos dataframes df_userfinal y df_itemsfinal

#Unimos los dos dataframes df_userfinal y df_itemsfinal en uno solo llamado df_useritems con merge

df_useritems = pd.merge(df_userfinal, df_itemsfinal, left_on='user_id', right_on='user_id')


In [56]:
#Lectura del dataframe

df_useritems

,user_id,user_url_x,funny,posted,last_edited,item_id_x,helpful,recommend,sentiment_analysis,items_count,steam_id,user_url_y,item_id_y,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,2,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6.0,0.0
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,2,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6.0,0.0
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,2,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6.0,0.0
3,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,2,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6.0,0.0
4,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,2,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7817956,76561198312638244,http://steamcommunity.com/profiles/76561198312...,,Posted June 2.,,209870,0 of 3 people (0%) found this review helpful,True,1,36,76561198312638244,http://steamcommunity.com/profiles/76561198312...,3990,Sid Meier's Civilization IV: Warlords,0.0,0.0
7817957,76561198312638244,http://steamcommunity.com/profiles/76561198312...,,Posted June 2.,,209870,0 of 3 people (0%) found this review helpful,True,1,36,76561198312638244,http://steamcommunity.com/profiles/76561198312...,3990,Sid Meier's Civilization IV: Warlords,0.0,0.0
7817958,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,,Posted April 19.,Last edited May 20.,437220,0 of 1 people (0%) found this review helpful,True,1,0,76561198313816521,http://steamcommunity.com/id/LydiaMorley,34450,Sid Meier's Civilization IV: Warlords,0.0,0.0
7817959,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,,Posted April 19.,Last edited May 20.,437220,0 of 1 people (0%) found this review helpful,True,1,0,76561198313816521,http://steamcommunity.com/id/LydiaMorley,34450,Sid Meier's Civilization IV: Warlords,0.0,0.0


In [57]:
#Borramos las columnas user_url_y y user_url_x

df_useritems = df_useritems.drop(['user_url_y'], axis=1)
df_useritems = df_useritems.drop(['user_url_x'], axis=1)

In [58]:
#Lectura del dataframe

df_useritems

,user_id,funny,posted,last_edited,item_id_x,helpful,recommend,sentiment_analysis,items_count,steam_id,item_id_y,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,2,277,76561197970982479,10,Counter-Strike,6.0,0.0
1,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,2,277,76561197970982479,10,Counter-Strike,6.0,0.0
2,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,2,277,76561197970982479,10,Counter-Strike,6.0,0.0
3,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,2,277,76561197970982479,10,Counter-Strike,6.0,0.0
4,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,2,277,76561197970982479,10,Counter-Strike,6.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7817956,76561198312638244,,Posted June 2.,,209870,0 of 3 people (0%) found this review helpful,True,1,36,76561198312638244,3990,Sid Meier's Civilization IV: Warlords,0.0,0.0
7817957,76561198312638244,,Posted June 2.,,209870,0 of 3 people (0%) found this review helpful,True,1,36,76561198312638244,3990,Sid Meier's Civilization IV: Warlords,0.0,0.0
7817958,LydiaMorley,,Posted April 19.,Last edited May 20.,437220,0 of 1 people (0%) found this review helpful,True,1,0,76561198313816521,34450,Sid Meier's Civilization IV: Warlords,0.0,0.0
7817959,LydiaMorley,,Posted April 19.,Last edited May 20.,437220,0 of 1 people (0%) found this review helpful,True,1,0,76561198313816521,34450,Sid Meier's Civilization IV: Warlords,0.0,0.0


In [59]:
df_useritems.head(5)

,user_id,funny,posted,last_edited,item_id_x,helpful,recommend,sentiment_analysis,items_count,steam_id,item_id_y,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,2,277,76561197970982479,10,Counter-Strike,6.0,0.0
1,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,2,277,76561197970982479,10,Counter-Strike,6.0,0.0
2,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,2,277,76561197970982479,10,Counter-Strike,6.0,0.0
3,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,2,277,76561197970982479,10,Counter-Strike,6.0,0.0
4,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,2,277,76561197970982479,10,Counter-Strike,6.0,0.0


In [60]:
#Unimos los dos dataframes df_useritems y df_games en uno solo llamado df_useritemsgames con merge usando la columna item_id_y y id de cada df respectivamente

df_useritemsgames = pd.merge(df_useritems, df_games, left_on='item_id_y', right_on='id')

In [61]:
#Lectura del dataframe

df_useritemsgames

,user_id,funny,posted,last_edited,item_id_x,helpful,recommend,sentiment_analysis,items_count,steam_id,...,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,year
0,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,2,277,76561197970982479,...,http://store.steampowered.com/app/10/CounterSt...,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...",http://steamcommunity.com/app/10/reviews/?brow...,"[Multi-player, Valve Anti-Cheat enabled]",9.99,False,10,Valve,2000
1,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,2,277,76561197970982479,...,http://store.steampowered.com/app/10/CounterSt...,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...",http://steamcommunity.com/app/10/reviews/?brow...,"[Multi-player, Valve Anti-Cheat enabled]",9.99,False,10,Valve,2000
2,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,2,277,76561197970982479,...,http://store.steampowered.com/app/10/CounterSt...,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...",http://steamcommunity.com/app/10/reviews/?brow...,"[Multi-player, Valve Anti-Cheat enabled]",9.99,False,10,Valve,2000
3,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,2,277,76561197970982479,...,http://store.steampowered.com/app/10/CounterSt...,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...",http://steamcommunity.com/app/10/reviews/?brow...,"[Multi-player, Valve Anti-Cheat enabled]",9.99,False,10,Valve,2000
4,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,2,277,76561197970982479,...,http://store.steampowered.com/app/10/CounterSt...,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...",http://steamcommunity.com/app/10/reviews/?brow...,"[Multi-player, Valve Anti-Cheat enabled]",9.99,False,10,Valve,2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6084698,cyvoxfromyoutube,,"Posted August 25, 2014.",,204360,No ratings yet,True,2,16,76561198185977117,...,http://store.steampowered.com/app/331390/Winge...,2014-11-07,"[Strategy, Anime, RPG, Indie, Tower Defense, F...",http://steamcommunity.com/app/331390/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",14.99,False,331390,WINGED SAKURA GAMES,2014
6084699,cyvoxfromyoutube,,"Posted August 25, 2014.",,204360,No ratings yet,True,2,16,76561198185977117,...,http://store.steampowered.com/app/331390/Winge...,2014-11-07,"[Strategy, Anime, RPG, Indie, Tower Defense, F...",http://steamcommunity.com/app/331390/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",14.99,False,331390,WINGED SAKURA GAMES,2014
6084700,cyvoxfromyoutube,,"Posted August 25, 2014.",,204360,No ratings yet,True,2,16,76561198185977117,...,http://store.steampowered.com/app/331390/Winge...,2014-11-07,"[Strategy, Anime, RPG, Indie, Tower Defense, F...",http://steamcommunity.com/app/331390/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",14.99,False,331390,WINGED SAKURA GAMES,2014
6084701,cyvoxfromyoutube,,"Posted August 25, 2014.",,204360,No ratings yet,True,2,16,76561198185977117,...,http://store.steampowered.com/app/331390/Winge...,2014-11-07,"[Strategy, Anime, RPG, Indie, Tower Defense, F...",http://steamcommunity.com/app/331390/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",14.99,False,331390,WINGED SAKURA GAMES,2014


In [62]:
df_useritemsgames.head(5)

,user_id,funny,posted,last_edited,item_id_x,helpful,recommend,sentiment_analysis,items_count,steam_id,...,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,year
0,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,2,277,76561197970982479,...,http://store.steampowered.com/app/10/CounterSt...,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...",http://steamcommunity.com/app/10/reviews/?brow...,"[Multi-player, Valve Anti-Cheat enabled]",9.99,False,10,Valve,2000
1,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,2,277,76561197970982479,...,http://store.steampowered.com/app/10/CounterSt...,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...",http://steamcommunity.com/app/10/reviews/?brow...,"[Multi-player, Valve Anti-Cheat enabled]",9.99,False,10,Valve,2000
2,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,2,277,76561197970982479,...,http://store.steampowered.com/app/10/CounterSt...,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...",http://steamcommunity.com/app/10/reviews/?brow...,"[Multi-player, Valve Anti-Cheat enabled]",9.99,False,10,Valve,2000
3,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,2,277,76561197970982479,...,http://store.steampowered.com/app/10/CounterSt...,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...",http://steamcommunity.com/app/10/reviews/?brow...,"[Multi-player, Valve Anti-Cheat enabled]",9.99,False,10,Valve,2000
4,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,2,277,76561197970982479,...,http://store.steampowered.com/app/10/CounterSt...,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...",http://steamcommunity.com/app/10/reviews/?brow...,"[Multi-player, Valve Anti-Cheat enabled]",9.99,False,10,Valve,2000


In [63]:
df_useritemsgames.columns

Index(['user_id', 'funny', 'posted', 'last_edited', 'item_id_x', 'helpful',
       'recommend', 'sentiment_analysis', 'items_count', 'steam_id',
       'item_id_y', 'item_name', 'playtime_forever', 'playtime_2weeks',
       'publisher', 'genres', 'title', 'url', 'release_date', 'tags',
       'reviews_url', 'specs', 'price', 'early_access', 'id', 'developer',
       'year'],
      dtype='object')

In [64]:
#Lectura del dataframe

df_useritemsgames

,user_id,funny,posted,last_edited,item_id_x,helpful,recommend,sentiment_analysis,items_count,steam_id,...,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,year
0,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,2,277,76561197970982479,...,http://store.steampowered.com/app/10/CounterSt...,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...",http://steamcommunity.com/app/10/reviews/?brow...,"[Multi-player, Valve Anti-Cheat enabled]",9.99,False,10,Valve,2000
1,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,2,277,76561197970982479,...,http://store.steampowered.com/app/10/CounterSt...,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...",http://steamcommunity.com/app/10/reviews/?brow...,"[Multi-player, Valve Anti-Cheat enabled]",9.99,False,10,Valve,2000
2,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,2,277,76561197970982479,...,http://store.steampowered.com/app/10/CounterSt...,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...",http://steamcommunity.com/app/10/reviews/?brow...,"[Multi-player, Valve Anti-Cheat enabled]",9.99,False,10,Valve,2000
3,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,2,277,76561197970982479,...,http://store.steampowered.com/app/10/CounterSt...,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...",http://steamcommunity.com/app/10/reviews/?brow...,"[Multi-player, Valve Anti-Cheat enabled]",9.99,False,10,Valve,2000
4,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,2,277,76561197970982479,...,http://store.steampowered.com/app/10/CounterSt...,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...",http://steamcommunity.com/app/10/reviews/?brow...,"[Multi-player, Valve Anti-Cheat enabled]",9.99,False,10,Valve,2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6084698,cyvoxfromyoutube,,"Posted August 25, 2014.",,204360,No ratings yet,True,2,16,76561198185977117,...,http://store.steampowered.com/app/331390/Winge...,2014-11-07,"[Strategy, Anime, RPG, Indie, Tower Defense, F...",http://steamcommunity.com/app/331390/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",14.99,False,331390,WINGED SAKURA GAMES,2014
6084699,cyvoxfromyoutube,,"Posted August 25, 2014.",,204360,No ratings yet,True,2,16,76561198185977117,...,http://store.steampowered.com/app/331390/Winge...,2014-11-07,"[Strategy, Anime, RPG, Indie, Tower Defense, F...",http://steamcommunity.com/app/331390/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",14.99,False,331390,WINGED SAKURA GAMES,2014
6084700,cyvoxfromyoutube,,"Posted August 25, 2014.",,204360,No ratings yet,True,2,16,76561198185977117,...,http://store.steampowered.com/app/331390/Winge...,2014-11-07,"[Strategy, Anime, RPG, Indie, Tower Defense, F...",http://steamcommunity.com/app/331390/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",14.99,False,331390,WINGED SAKURA GAMES,2014
6084701,cyvoxfromyoutube,,"Posted August 25, 2014.",,204360,No ratings yet,True,2,16,76561198185977117,...,http://store.steampowered.com/app/331390/Winge...,2014-11-07,"[Strategy, Anime, RPG, Indie, Tower Defense, F...",http://steamcommunity.com/app/331390/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",14.99,False,331390,WINGED SAKURA GAMES,2014


In [65]:
#Borramos la columna reviews_url

df_useritemsgames = df_useritemsgames.drop(['reviews_url'], axis=1)

In [66]:
#Vemos los unique de early_access

print(df_useritemsgames['early_access'].unique())

[False True]


In [67]:
#Borramos la columna url

df_useritemsgames = df_useritemsgames.drop(['url'], axis=1)

#Borramos la columna helpful

df_useritemsgames = df_useritemsgames.drop(['helpful'], axis=1)

In [68]:
#Lectura del dataframe

df_useritemsgames

,user_id,funny,posted,last_edited,item_id_x,recommend,sentiment_analysis,items_count,steam_id,item_id_y,...,genres,title,release_date,tags,specs,price,early_access,id,developer,year
0,76561197970982479,,"Posted November 5, 2011.",,1250,True,2,277,76561197970982479,10,...,[Action],Counter-Strike,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...","[Multi-player, Valve Anti-Cheat enabled]",9.99,False,10,Valve,2000
1,76561197970982479,,"Posted November 5, 2011.",,1250,True,2,277,76561197970982479,10,...,[Action],Counter-Strike,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...","[Multi-player, Valve Anti-Cheat enabled]",9.99,False,10,Valve,2000
2,76561197970982479,,"Posted November 5, 2011.",,1250,True,2,277,76561197970982479,10,...,[Action],Counter-Strike,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...","[Multi-player, Valve Anti-Cheat enabled]",9.99,False,10,Valve,2000
3,76561197970982479,,"Posted November 5, 2011.",,1250,True,2,277,76561197970982479,10,...,[Action],Counter-Strike,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...","[Multi-player, Valve Anti-Cheat enabled]",9.99,False,10,Valve,2000
4,76561197970982479,,"Posted November 5, 2011.",,1250,True,2,277,76561197970982479,10,...,[Action],Counter-Strike,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...","[Multi-player, Valve Anti-Cheat enabled]",9.99,False,10,Valve,2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6084698,cyvoxfromyoutube,,"Posted August 25, 2014.",,204360,True,2,16,76561198185977117,331390,...,"[Indie, RPG, Strategy]",Winged Sakura: Mindy's Arc,2014-11-07,"[Strategy, Anime, RPG, Indie, Tower Defense, F...","[Single-player, Steam Achievements, Steam Trad...",14.99,False,331390,WINGED SAKURA GAMES,2014
6084699,cyvoxfromyoutube,,"Posted August 25, 2014.",,204360,True,2,16,76561198185977117,331390,...,"[Indie, RPG, Strategy]",Winged Sakura: Mindy's Arc,2014-11-07,"[Strategy, Anime, RPG, Indie, Tower Defense, F...","[Single-player, Steam Achievements, Steam Trad...",14.99,False,331390,WINGED SAKURA GAMES,2014
6084700,cyvoxfromyoutube,,"Posted August 25, 2014.",,204360,True,2,16,76561198185977117,331390,...,"[Indie, RPG, Strategy]",Winged Sakura: Mindy's Arc,2014-11-07,"[Strategy, Anime, RPG, Indie, Tower Defense, F...","[Single-player, Steam Achievements, Steam Trad...",14.99,False,331390,WINGED SAKURA GAMES,2014
6084701,cyvoxfromyoutube,,"Posted August 25, 2014.",,204360,True,2,16,76561198185977117,331390,...,"[Indie, RPG, Strategy]",Winged Sakura: Mindy's Arc,2014-11-07,"[Strategy, Anime, RPG, Indie, Tower Defense, F...","[Single-player, Steam Achievements, Steam Trad...",14.99,False,331390,WINGED SAKURA GAMES,2014


In [69]:
#Se borra la columna item_id_y porque es la misma que id y se renombra la columna item_id_x a item_id

df_useritemsgames = df_useritemsgames.drop(['item_id_y'], axis=1)
df_useritemsgames = df_useritemsgames.rename(columns={'item_id_x': 'item_id'})

#Acotacion item_id_x es el id del usuario(Es decir el como el nro de id de usuario de steam) y item_id_y o id es el id del juego

In [70]:
#Lectura del dataframe

df_useritemsgames.head(5)

,user_id,funny,posted,last_edited,item_id,recommend,sentiment_analysis,items_count,steam_id,item_name,...,genres,title,release_date,tags,specs,price,early_access,id,developer,year
0,76561197970982479,,"Posted November 5, 2011.",,1250,True,2,277,76561197970982479,Counter-Strike,...,[Action],Counter-Strike,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...","[Multi-player, Valve Anti-Cheat enabled]",9.99,False,10,Valve,2000
1,76561197970982479,,"Posted November 5, 2011.",,1250,True,2,277,76561197970982479,Counter-Strike,...,[Action],Counter-Strike,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...","[Multi-player, Valve Anti-Cheat enabled]",9.99,False,10,Valve,2000
2,76561197970982479,,"Posted November 5, 2011.",,1250,True,2,277,76561197970982479,Counter-Strike,...,[Action],Counter-Strike,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...","[Multi-player, Valve Anti-Cheat enabled]",9.99,False,10,Valve,2000
3,76561197970982479,,"Posted November 5, 2011.",,1250,True,2,277,76561197970982479,Counter-Strike,...,[Action],Counter-Strike,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...","[Multi-player, Valve Anti-Cheat enabled]",9.99,False,10,Valve,2000
4,76561197970982479,,"Posted November 5, 2011.",,1250,True,2,277,76561197970982479,Counter-Strike,...,[Action],Counter-Strike,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...","[Multi-player, Valve Anti-Cheat enabled]",9.99,False,10,Valve,2000


In [71]:
#Cambiamos los valores str de la columna prince a float

df_useritemsgames['price'] = df_useritemsgames['price'].astype(float)

In [72]:
#Vemos la columna price

df_useritemsgames['price']

0           9.99
1           9.99
2           9.99
3           9.99
4           9.99
           ...  
6084698    14.99
6084699    14.99
6084700    14.99
6084701    14.99
6084702    14.99
Name: price, Length: 6084703, dtype: float64

In [73]:
#Cambiamos los valores str de la columna item_id a int y borramos los NaNs
df_useritemsgames.dropna(subset=['item_id'], inplace=True)
df_useritemsgames['item_id'] = df_useritemsgames['item_id'].astype(int)


In [74]:
#Vemos la columna item_id

df_useritemsgames['item_id']

0            1250
1            1250
2            1250
3            1250
4            1250
            ...  
6084698    204360
6084699    204360
6084700    204360
6084701    204360
6084702    204360
Name: item_id, Length: 6081633, dtype: int32

In [75]:
'''
Creamos la columna MoneySpend siendo este obtenido verificando el user_id y corroborando
la cantidad de items que ha comprado cada user_id y el precio de cada item multiplicando
la cantidad de items por el precio de cada item
'''

df_useritemsgames['MoneySpend'] = df_useritemsgames.groupby('user_id')['price'].transform('sum')




In [76]:
df_useritemsgames

,user_id,funny,posted,last_edited,item_id,recommend,sentiment_analysis,items_count,steam_id,item_name,...,title,release_date,tags,specs,price,early_access,id,developer,year,MoneySpend
0,76561197970982479,,"Posted November 5, 2011.",,1250,True,2,277,76561197970982479,Counter-Strike,...,Counter-Strike,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...","[Multi-player, Valve Anti-Cheat enabled]",9.99,False,10,Valve,2000,8301.69
1,76561197970982479,,"Posted November 5, 2011.",,1250,True,2,277,76561197970982479,Counter-Strike,...,Counter-Strike,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...","[Multi-player, Valve Anti-Cheat enabled]",9.99,False,10,Valve,2000,8301.69
2,76561197970982479,,"Posted November 5, 2011.",,1250,True,2,277,76561197970982479,Counter-Strike,...,Counter-Strike,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...","[Multi-player, Valve Anti-Cheat enabled]",9.99,False,10,Valve,2000,8301.69
3,76561197970982479,,"Posted November 5, 2011.",,1250,True,2,277,76561197970982479,Counter-Strike,...,Counter-Strike,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...","[Multi-player, Valve Anti-Cheat enabled]",9.99,False,10,Valve,2000,8301.69
4,76561197970982479,,"Posted November 5, 2011.",,1250,True,2,277,76561197970982479,Counter-Strike,...,Counter-Strike,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...","[Multi-player, Valve Anti-Cheat enabled]",9.99,False,10,Valve,2000,8301.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6084698,cyvoxfromyoutube,,"Posted August 25, 2014.",,204360,True,2,16,76561198185977117,Winged Sakura: Mindy's Arc,...,Winged Sakura: Mindy's Arc,2014-11-07,"[Strategy, Anime, RPG, Indie, Tower Defense, F...","[Single-player, Steam Achievements, Steam Trad...",14.99,False,331390,WINGED SAKURA GAMES,2014,719.52
6084699,cyvoxfromyoutube,,"Posted August 25, 2014.",,204360,True,2,16,76561198185977117,Winged Sakura: Mindy's Arc,...,Winged Sakura: Mindy's Arc,2014-11-07,"[Strategy, Anime, RPG, Indie, Tower Defense, F...","[Single-player, Steam Achievements, Steam Trad...",14.99,False,331390,WINGED SAKURA GAMES,2014,719.52
6084700,cyvoxfromyoutube,,"Posted August 25, 2014.",,204360,True,2,16,76561198185977117,Winged Sakura: Mindy's Arc,...,Winged Sakura: Mindy's Arc,2014-11-07,"[Strategy, Anime, RPG, Indie, Tower Defense, F...","[Single-player, Steam Achievements, Steam Trad...",14.99,False,331390,WINGED SAKURA GAMES,2014,719.52
6084701,cyvoxfromyoutube,,"Posted August 25, 2014.",,204360,True,2,16,76561198185977117,Winged Sakura: Mindy's Arc,...,Winged Sakura: Mindy's Arc,2014-11-07,"[Strategy, Anime, RPG, Indie, Tower Defense, F...","[Single-player, Steam Achievements, Steam Trad...",14.99,False,331390,WINGED SAKURA GAMES,2014,719.52


In [77]:
#Vemos un indice de las columnas

df_useritemsgames.columns

Index(['user_id', 'funny', 'posted', 'last_edited', 'item_id', 'recommend',
       'sentiment_analysis', 'items_count', 'steam_id', 'item_name',
       'playtime_forever', 'playtime_2weeks', 'publisher', 'genres', 'title',
       'release_date', 'tags', 'specs', 'price', 'early_access', 'id',
       'developer', 'year', 'MoneySpend'],
      dtype='object')

In [78]:
#Vemos las columnas 1,3 y 22

df_useritemsgames.iloc[:,[1,3,22]]

#vemos los tipos de datos que manejan esas columnas

df_useritemsgames.iloc[:,[1,3,22]].dtypes

funny          object
last_edited    object
year           object
dtype: object

In [79]:
#Vemos los unique de la columna year

print(df_useritemsgames['year'].unique())

['2000' '1999' '2003' '2010' '2004' '2005' '2006' '2007' 'Jun ' '2016'
 '1997' '2008' '2009' '2002' '2011' '2014' '2012' '2013' '2015' '1998'
 '2001' '1994' '1995' '1996' '2017' '1993' '1990' '1991' '1992' '1988'
 'Jan ' 'Jul ' '1983' '1984' 'Feb ' 'Oct ' 'May ' '1987' 'Sep ']


In [80]:
#Sustituimos "Jan","Jun","Dec","Nov","Oct","Sep","Aug","Jul","May","Apr","Mar","Feb" por 0

df_useritemsgames['year'] = df_useritemsgames['year'].replace(['Jan ' ,'Jun ','Dec ','Nov ','Oct ','Sep ','Aug ','Jul ','May ','Apr ','Mar ','Feb ','Jun '], '0')

In [81]:
#Vemos los unique de la columna year

print(df_useritemsgames['year'].unique())

['2000' '1999' '2003' '2010' '2004' '2005' '2006' '2007' '0' '2016' '1997'
 '2008' '2009' '2002' '2011' '2014' '2012' '2013' '2015' '1998' '2001'
 '1994' '1995' '1996' '2017' '1993' '1990' '1991' '1992' '1988' '1983'
 '1984' '1987']


In [82]:
#Cambiamos los valores str de la columna year a int

df_useritemsgames['year'] = df_useritemsgames['year'].astype(int)

In [83]:
#Lectura del dataframe

df_useritemsgames

,user_id,funny,posted,last_edited,item_id,recommend,sentiment_analysis,items_count,steam_id,item_name,...,title,release_date,tags,specs,price,early_access,id,developer,year,MoneySpend
0,76561197970982479,,"Posted November 5, 2011.",,1250,True,2,277,76561197970982479,Counter-Strike,...,Counter-Strike,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...","[Multi-player, Valve Anti-Cheat enabled]",9.99,False,10,Valve,2000,8301.69
1,76561197970982479,,"Posted November 5, 2011.",,1250,True,2,277,76561197970982479,Counter-Strike,...,Counter-Strike,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...","[Multi-player, Valve Anti-Cheat enabled]",9.99,False,10,Valve,2000,8301.69
2,76561197970982479,,"Posted November 5, 2011.",,1250,True,2,277,76561197970982479,Counter-Strike,...,Counter-Strike,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...","[Multi-player, Valve Anti-Cheat enabled]",9.99,False,10,Valve,2000,8301.69
3,76561197970982479,,"Posted November 5, 2011.",,1250,True,2,277,76561197970982479,Counter-Strike,...,Counter-Strike,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...","[Multi-player, Valve Anti-Cheat enabled]",9.99,False,10,Valve,2000,8301.69
4,76561197970982479,,"Posted November 5, 2011.",,1250,True,2,277,76561197970982479,Counter-Strike,...,Counter-Strike,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...","[Multi-player, Valve Anti-Cheat enabled]",9.99,False,10,Valve,2000,8301.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6084698,cyvoxfromyoutube,,"Posted August 25, 2014.",,204360,True,2,16,76561198185977117,Winged Sakura: Mindy's Arc,...,Winged Sakura: Mindy's Arc,2014-11-07,"[Strategy, Anime, RPG, Indie, Tower Defense, F...","[Single-player, Steam Achievements, Steam Trad...",14.99,False,331390,WINGED SAKURA GAMES,2014,719.52
6084699,cyvoxfromyoutube,,"Posted August 25, 2014.",,204360,True,2,16,76561198185977117,Winged Sakura: Mindy's Arc,...,Winged Sakura: Mindy's Arc,2014-11-07,"[Strategy, Anime, RPG, Indie, Tower Defense, F...","[Single-player, Steam Achievements, Steam Trad...",14.99,False,331390,WINGED SAKURA GAMES,2014,719.52
6084700,cyvoxfromyoutube,,"Posted August 25, 2014.",,204360,True,2,16,76561198185977117,Winged Sakura: Mindy's Arc,...,Winged Sakura: Mindy's Arc,2014-11-07,"[Strategy, Anime, RPG, Indie, Tower Defense, F...","[Single-player, Steam Achievements, Steam Trad...",14.99,False,331390,WINGED SAKURA GAMES,2014,719.52
6084701,cyvoxfromyoutube,,"Posted August 25, 2014.",,204360,True,2,16,76561198185977117,Winged Sakura: Mindy's Arc,...,Winged Sakura: Mindy's Arc,2014-11-07,"[Strategy, Anime, RPG, Indie, Tower Defense, F...","[Single-player, Steam Achievements, Steam Trad...",14.99,False,331390,WINGED SAKURA GAMES,2014,719.52


In [84]:
#ver unique de release_date y last_edited

print(df_useritemsgames['release_date'].unique())


['2000-11-01' '1999-04-01' '2003-05-01' ... '2006-10-13' '2005-09-16'
 '2015-05-11']


In [85]:
# Eliminar "Posted" y espacios adicionales de la columna "posted"
df_useritemsgames['posted'] = df_useritemsgames['posted'].str.replace('Posted', '').str.strip()

In [86]:
#Lectura del dataframe

df_useritemsgames

,user_id,funny,posted,last_edited,item_id,recommend,sentiment_analysis,items_count,steam_id,item_name,...,title,release_date,tags,specs,price,early_access,id,developer,year,MoneySpend
0,76561197970982479,,"November 5, 2011.",,1250,True,2,277,76561197970982479,Counter-Strike,...,Counter-Strike,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...","[Multi-player, Valve Anti-Cheat enabled]",9.99,False,10,Valve,2000,8301.69
1,76561197970982479,,"November 5, 2011.",,1250,True,2,277,76561197970982479,Counter-Strike,...,Counter-Strike,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...","[Multi-player, Valve Anti-Cheat enabled]",9.99,False,10,Valve,2000,8301.69
2,76561197970982479,,"November 5, 2011.",,1250,True,2,277,76561197970982479,Counter-Strike,...,Counter-Strike,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...","[Multi-player, Valve Anti-Cheat enabled]",9.99,False,10,Valve,2000,8301.69
3,76561197970982479,,"November 5, 2011.",,1250,True,2,277,76561197970982479,Counter-Strike,...,Counter-Strike,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...","[Multi-player, Valve Anti-Cheat enabled]",9.99,False,10,Valve,2000,8301.69
4,76561197970982479,,"November 5, 2011.",,1250,True,2,277,76561197970982479,Counter-Strike,...,Counter-Strike,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...","[Multi-player, Valve Anti-Cheat enabled]",9.99,False,10,Valve,2000,8301.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6084698,cyvoxfromyoutube,,"August 25, 2014.",,204360,True,2,16,76561198185977117,Winged Sakura: Mindy's Arc,...,Winged Sakura: Mindy's Arc,2014-11-07,"[Strategy, Anime, RPG, Indie, Tower Defense, F...","[Single-player, Steam Achievements, Steam Trad...",14.99,False,331390,WINGED SAKURA GAMES,2014,719.52
6084699,cyvoxfromyoutube,,"August 25, 2014.",,204360,True,2,16,76561198185977117,Winged Sakura: Mindy's Arc,...,Winged Sakura: Mindy's Arc,2014-11-07,"[Strategy, Anime, RPG, Indie, Tower Defense, F...","[Single-player, Steam Achievements, Steam Trad...",14.99,False,331390,WINGED SAKURA GAMES,2014,719.52
6084700,cyvoxfromyoutube,,"August 25, 2014.",,204360,True,2,16,76561198185977117,Winged Sakura: Mindy's Arc,...,Winged Sakura: Mindy's Arc,2014-11-07,"[Strategy, Anime, RPG, Indie, Tower Defense, F...","[Single-player, Steam Achievements, Steam Trad...",14.99,False,331390,WINGED SAKURA GAMES,2014,719.52
6084701,cyvoxfromyoutube,,"August 25, 2014.",,204360,True,2,16,76561198185977117,Winged Sakura: Mindy's Arc,...,Winged Sakura: Mindy's Arc,2014-11-07,"[Strategy, Anime, RPG, Indie, Tower Defense, F...","[Single-player, Steam Achievements, Steam Trad...",14.99,False,331390,WINGED SAKURA GAMES,2014,719.52


In [87]:
#Creamos una funcion para transformar la columna posted a datetime teniendo en cuenta que el contenido que tiene es "November 5, 2011." por ejemplo y queremos en YYYY-MM-DD

def extract_date(text):
    match = re.search(r'\b(\w+\s\d+,\s\d{4})\b', text)
    if match:
        return pd.to_datetime(match.group(1), format='%B %d, %Y')
    else:
        return None


In [88]:
#Aplicamos la funcion a la columna posted

df_useritemsgames['posted'] = df_useritemsgames['posted'].apply(extract_date)

In [89]:
#Lectura del dataframe

df_useritemsgames

,user_id,funny,posted,last_edited,item_id,recommend,sentiment_analysis,items_count,steam_id,item_name,...,title,release_date,tags,specs,price,early_access,id,developer,year,MoneySpend
0,76561197970982479,,2011-11-05,,1250,True,2,277,76561197970982479,Counter-Strike,...,Counter-Strike,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...","[Multi-player, Valve Anti-Cheat enabled]",9.99,False,10,Valve,2000,8301.69
1,76561197970982479,,2011-11-05,,1250,True,2,277,76561197970982479,Counter-Strike,...,Counter-Strike,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...","[Multi-player, Valve Anti-Cheat enabled]",9.99,False,10,Valve,2000,8301.69
2,76561197970982479,,2011-11-05,,1250,True,2,277,76561197970982479,Counter-Strike,...,Counter-Strike,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...","[Multi-player, Valve Anti-Cheat enabled]",9.99,False,10,Valve,2000,8301.69
3,76561197970982479,,2011-11-05,,1250,True,2,277,76561197970982479,Counter-Strike,...,Counter-Strike,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...","[Multi-player, Valve Anti-Cheat enabled]",9.99,False,10,Valve,2000,8301.69
4,76561197970982479,,2011-11-05,,1250,True,2,277,76561197970982479,Counter-Strike,...,Counter-Strike,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...","[Multi-player, Valve Anti-Cheat enabled]",9.99,False,10,Valve,2000,8301.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6084698,cyvoxfromyoutube,,2014-08-25,,204360,True,2,16,76561198185977117,Winged Sakura: Mindy's Arc,...,Winged Sakura: Mindy's Arc,2014-11-07,"[Strategy, Anime, RPG, Indie, Tower Defense, F...","[Single-player, Steam Achievements, Steam Trad...",14.99,False,331390,WINGED SAKURA GAMES,2014,719.52
6084699,cyvoxfromyoutube,,2014-08-25,,204360,True,2,16,76561198185977117,Winged Sakura: Mindy's Arc,...,Winged Sakura: Mindy's Arc,2014-11-07,"[Strategy, Anime, RPG, Indie, Tower Defense, F...","[Single-player, Steam Achievements, Steam Trad...",14.99,False,331390,WINGED SAKURA GAMES,2014,719.52
6084700,cyvoxfromyoutube,,2014-08-25,,204360,True,2,16,76561198185977117,Winged Sakura: Mindy's Arc,...,Winged Sakura: Mindy's Arc,2014-11-07,"[Strategy, Anime, RPG, Indie, Tower Defense, F...","[Single-player, Steam Achievements, Steam Trad...",14.99,False,331390,WINGED SAKURA GAMES,2014,719.52
6084701,cyvoxfromyoutube,,2014-08-25,,204360,True,2,16,76561198185977117,Winged Sakura: Mindy's Arc,...,Winged Sakura: Mindy's Arc,2014-11-07,"[Strategy, Anime, RPG, Indie, Tower Defense, F...","[Single-player, Steam Achievements, Steam Trad...",14.99,False,331390,WINGED SAKURA GAMES,2014,719.52


In [90]:


df_useritemsgames.dtypes

user_id                       object
funny                         object
posted                datetime64[ns]
last_edited                   object
item_id                        int32
recommend                     object
sentiment_analysis             int64
items_count                    int64
steam_id                      object
item_name                     object
playtime_forever             float64
playtime_2weeks              float64
publisher                     object
genres                        object
title                         object
release_date                  object
tags                          object
specs                         object
price                        float64
early_access                  object
id                            object
developer                     object
year                           int32
MoneySpend                   float64
dtype: object

In [91]:
#Vemos tipo de datos de la columna genre

df_useritemsgames['genres'].dtypes

dtype('O')

In [92]:
# Creamos la columna contenido_free la cual sera de valor true si el contenido es free y false si no lo es
# tendra como parametro la columna price y si price es igual a 0.00 sera true y si no false


df_useritemsgames['contenido_free'] = df_useritemsgames['price'].apply(lambda x: True if x == 0.00 else False)


In [99]:
#Lectura del dataframe

df_useritemsgames.sample(5)

,user_id,funny,posted,last_edited,item_id,recommend,sentiment_analysis,items_count,steam_id,item_name,...,release_date,tags,specs,price,early_access,id,developer,year,MoneySpend,contenido_free
3586861,Flakerinos,,2014-03-13,"Last edited March 13, 2014.",211820,True,1,185,76561198027966730,Clicker Heroes,...,2015-05-13,"[Clicker, Free to Play, Casual, Singleplayer, ...","[Single-player, Steam Achievements, Steam Trad...",0.00,False,363970,Playsaurus,2015,0.00,True
331416,mrtenacity99,,NaT,Last edited May 22.,375230,False,0,30,76561198071567822,Call of Duty: Modern Warfare 2,...,2009-11-12,"[Action, FPS, Multiplayer, Shooter, First-Pers...","[Single-player, Multi-player, Co-op, Steam Ach...",19.99,False,10180,"Infinity Ward,Aspyr (Mac)",2009,599.70,False
2790632,sdq101,,2015-07-16,,273110,False,0,132,76561198014424015,XCOM: Enemy Unknown,...,2012-10-08,"[Turn-Based Strategy, Tactical, Strategy, Sci-...","[Single-player, Multi-player, Cross-Platform M...",29.99,False,200510,"Firaxis Games,Feral Interactive (Mac),Feral In...",2012,31669.44,False
72760,76561198160316683,,NaT,,412880,True,2,104,76561198160316683,Counter-Strike: Source,...,2004-11-01,"[Action, FPS, Multiplayer, Shooter, Team-Based...","[Multi-player, Cross-Platform Multiplayer, Ste...",19.99,False,240,Valve,2004,6236.88,False
553070,76561198060201321,,2014-12-28,"Last edited January 4, 2015.",236390,True,2,112,76561198060201321,The Binding of Isaac,...,2011-09-28,"[Rogue-like, Indie, Replay Value, Difficult, P...","[Single-player, Steam Achievements, Steam Trad...",4.99,False,113200,Edmund McMillen and Florian Himsl,2011,5588.80,False


In [104]:
#Vemos la columna genres

df_useritemsgames['genres']

0                        [Action]
1                        [Action]
2                        [Action]
3                        [Action]
4                        [Action]
                    ...          
6084698    [Indie, RPG, Strategy]
6084699    [Indie, RPG, Strategy]
6084700    [Indie, RPG, Strategy]
6084701    [Indie, RPG, Strategy]
6084702    [Indie, RPG, Strategy]
Name: genres, Length: 6081633, dtype: object

In [97]:
#Guardamos los cambios hechos en el dataframe df_useritemsgames en un csv llamado df_useritemsgames.csv

df_useritemsgames.to_csv('df_useritemsgames.csv', index=False)
